In [108]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import numpy as np
import random
import torch
import transformers
import torch.nn as nn
from transformers import AutoModel, BertTokenizer, BertForSequenceClassification, AutoTokenizer
from transformers import TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, f1_score
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
import os
from tqdm.notebook import tqdm

In [140]:
max_length = 32
path = os.getcwd()
#path = os.path.join(path, 'train_data')
path

'/content'

In [141]:
tokenizer = BertTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence", num_labels=2, output_attentions = False, output_hidden_states = False)
model = BertForSequenceClassification.from_pretrained("DeepPavlov/rubert-base-cased-sentence")
model

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-sentence and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [72]:
class CustomDataset(Dataset):
    def __init__(self, path, max_length, tokenizer):
        self._path = path
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.positive = 0
        self.negative = 0
        self.lines = []
        self.labels = []
        self.fonts = []
        self.bolds = []
        for output_file in os.listdir(path):
            file_dir = os.path.join(path, output_file)
            if not file_dir.endswith('.txt'):
                continue
            with open(file_dir, 'r', encoding='UTF-8') as f:
                for line in f.readlines():
                    line1, line2 = line.split('[font=')
                    self.lines.append(line1[:-1])
                    if line2[-3] == '1':
                        self.positive += 1
                        self.labels.append(1)
                    else:
                        self.negative += 1
                        self.labels.append(0)

                    line2 = line2.split(', ')
                    self.fonts.append(float(line2[0]))
                    self.bolds.append(int(line2[1][-1]))

    def __len__(self):
        return len(self.lines)

    def __getitem__(self, idx):
        if isinstance(idx, list):
            return [self._get_single_item(i) for i in idx]
        else:
            return self._get_single_item(idx)

    def _get_single_item(self, idx):
        return self.lines[idx]
#         encoding = self.tokenizer.encode_plus(
#             self.lines[idx],
#             add_special_tokens=True,
#             max_length=self.max_length,
#             padding='max_length',
#             truncation=True,
#             return_tensors='pt'
#         )
#         return {
#             'input_ids': encoding['input_ids'].flatten(),
#             'attention_mask': encoding['attention_mask'].flatten(),
#             'label': torch.tensor(self.labels[idx], dtype=torch.long),
# #             'font_size': torch.tensor(self.fonts[idx]),
# #             'bold': torch.tensor(self.bolds[idx])
#         }


In [ ]:
dataset = CustomDataset(path, max_length, tokenizer)

In [73]:
positive = 0
negative = 0
lines = []
labels = []
fonts = []
bolds = []
for output_file in os.listdir(path):
    file_dir = os.path.join(path, output_file)
    if not file_dir.endswith('.txt'):
        continue
    with open(file_dir, 'r', encoding='UTF-8') as f:
        for line in f.readlines():
            flag = 0
            line1, line2 = line.split('[font=')
            lines.append(line1[:-1])
            if line2[-3] == '1':
                positive += 3
                labels.append(1)
                labels.append(1)
                labels.append(1)
                lines.append(line1[:-1])
                lines.append(line1[:-1])
                flag = 1
            else:
                negative += 1
                labels.append(0)
            line2 = line2.split(', ')
            fonts.append(float(line2[0]))
            bolds.append(int(line2[1][-1]))
            if flag == 1:
                fonts.append(float(line2[0]))
                bolds.append(int(line2[1][-1]))
                fonts.append(float(line2[0]))
                bolds.append(int(line2[1][-1]))

In [75]:
negative

50739

In [76]:
dataset = pd.DataFrame({
    'target': labels,
    'text': lines,
})
dataset[dataset.target == 0] = dataset[(dataset.target == 0) & (dataset.text.apply(lambda x: len(x.split()) >= 4))]
dataset = dataset.dropna()
dataset.reset_index(drop=True, inplace=True)

df_target_0 = dataset[dataset['target'] == 0]

df_target_0_sampled = df_target_0.sample(frac=0.5, random_state=42)

# Drop the sampled rows from the original DataFrame
dataset = dataset.drop(df_target_0_sampled.index)

In [77]:
dataset

,target,text
1,0.0,(решение акционера - владельца
2,0.0,от «12» мая 2022 г. No 35)
5,0.0,ноября 2022 г. No 38);
6,1.0,I. Общие положения
7,1.0,I. Общие положения
...,...,...
38468,0.0,"неопределенности, в том числе снижение ликвидн..."
38469,0.0,"рынках капитала, изменчивость курса российског..."
38471,0.0,"перспективе представляется затруднительным, од..."
38474,0.0,"на то, какое влияние оказывают условия ведения..."


In [79]:
train_df, test_df = train_test_split(dataset, test_size=0.2, random_state=42, shuffle=True)
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)
train_df

,target,text
0,0.0,с ними ПО 5таг5Ку Рго. Реализована возможность
1,0.0,"32 Приобретение собственных акций (долей), 0 0"
2,0.0,акций Общества; генеральный директор
3,0.0,"доступности инфраструктуры, а в долгосрочной п..."
4,0.0,странице Единого платежного решения;
...,...,...
15980,0.0,получить высококачественную медицинскую помощь.
15981,1.0,Глава 7. Система управления
15982,0.0,1 2 3 4 5 6 7
15983,0.0,"Акции 67 0,01% 130 0,06% 63 0,04%"


In [80]:
train_text = train_df.text.astype('str')
train_labels = train_df.target
test_text = test_df.text.astype('str')
test_labels = test_df.target

In [81]:
train_text

,text
0,с ними ПО 5таг5Ку Рго. Реализована возможность
1,"32 Приобретение собственных акций (долей), 0 0"
2,акций Общества; генеральный директор
3,"доступности инфраструктуры, а в долгосрочной п..."
4,странице Единого платежного решения;
...,...
15980,получить высококачественную медицинскую помощь.
15981,Глава 7. Система управления
15982,1 2 3 4 5 6 7
15983,"Акции 67 0,01% 130 0,06% 63 0,04%"


In [142]:
tokens_train = tokenizer.batch_encode_plus(
    train_text.values,
    max_length = 32,
    padding = 'max_length',
    truncation = True
)
tokens_test = tokenizer.batch_encode_plus(
    test_text.values,
    max_length = 32,
    padding = 'max_length',
    truncation = True
)

In [143]:
class Data(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        if self.labels[idx] == 1.0:
            item["labels"] = torch.tensor([0.0, self.labels[idx]])
        else:
            item["labels"] = torch.tensor([1.0, 0.0])
        return item
    def __len__(self):
        return len(self.labels)

train_dataset = Data(tokens_train, train_labels)
test_dataset = Data(tokens_test, test_labels)

In [144]:
class_weights = compute_class_weight('balanced', classes=np.unique(train_labels), y=train_labels)
class_weights

array([0.54007028, 6.73903879])

In [152]:
class CustomTrainer(Trainer):
    def __init__(self, model, *args, cw=None, **kwargs):
        super().__init__(model, *args, **kwargs)
        self.cw = cw

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        if self.cw is not None:
            self.cw = torch.tensor(self.cw).to(model.device)
            loss_fct = nn.CrossEntropyLoss(weight=self.cw)
        else:
            loss_fct = nn.CrossEntropyLoss()
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [153]:
training_args = TrainingArguments(
    output_dir = './results', #Выходной каталог
    num_train_epochs = 10, #Кол-во эпох для обучения
    per_device_train_batch_size = 256, #Размер пакета для каждого устройства во время обучения
    per_device_eval_batch_size = 256, #Размер пакета для каждого устройства во время валидации
    weight_decay =0.01, #Понижение весов
    logging_dir = './logs', #Каталог для хранения журналов
    load_best_model_at_end = True, #Загружать ли лучшую модель после обучения
    learning_rate = 1e-5, #Скорость обучения
    eval_strategy ='epoch', #Валидация после каждой эпохи (можно сделать после конкретного кол-ва шагов)
    logging_strategy = 'epoch', #Логирование после каждой эпохи
    save_strategy = 'epoch', #Сохранение после каждой эпохи
    save_total_limit = 10,
    seed=21)

trainer = CustomTrainer(model=model,
                  tokenizer = tokenizer,
                  args = training_args,
                  train_dataset = train_dataset,
                  eval_dataset = train_dataset,
                  cw = class_weights)

In [154]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.371100,0.178494
2,0.144200,0.074122
3,0.074700,0.054437
4,0.050800,0.035399
5,0.040400,0.028055
6,0.031800,0.026366
7,0.028800,0.020827
8,0.025100,0.018916
9,0.021600,0.019831
10,0.020000,0.018946


<ipython-input-152-4e6143fa9021>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.cw = torch.tensor(self.cw).to(model.device)
<ipython-input-152-4e6143fa9021>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.cw = torch.tensor(self.cw).to(model.device)
<ipython-input-152-4e6143fa9021>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.cw = torch.tensor(self.cw).to(model.device)
<ipython-input-152-4e6143fa9021>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceT

TrainOutput(global_step=630, training_loss=0.0808597549559578, metrics={'train_runtime': 1547.7664, 'train_samples_per_second': 103.278, 'train_steps_per_second': 0.407, 'total_flos': 2628643887456000.0, 'train_loss': 0.0808597549559578, 'epoch': 10.0})

In [155]:
torch.save(model.state_dict(), 'bert.pt')

In [168]:
ans = []
with open('output51.txt', 'r') as f:
    i = 0
    for line in f.readlines():
        line, _ = line.split('[font=')
        input = tokenizer(line, return_tensors='pt', truncation=True, padding=True).to('cuda')
        with torch.no_grad():
            outputs = model(**input)
            preds = torch.softmax(outputs.logits, dim=-1)
            if preds[0][1] > 0.99:
                i += 1
                print(tokenizer.decode(input['input_ids'][0].tolist()))
                ans.append(tokenizer.decode(input['input_ids'][0].tolist()))
        if i == 30:
            break
with open('ans.txt', 'w') as f:
    f.write('\n'.join(ans))

[CLS] АУДИТОРСКОЕ ЗАКЛЮЧЕНИЕ НЕЗАВИСИМОГО АУДИТОРА [SEP]
[CLS] Мнение [SEP]
[CLS] Основание для выражепия мнения [SEP]
[CLS] АУДИТОР [SEP]
[CLS] Ключевые вопросы аудита [SEP]
[CLS] Ответственность руководства и членов совета директоров аудируемого лица за [SEP]
[CLS] годовую бухгалтерскую ( фипапсовую ) отчетность [SEP]
[CLS] деятельности. [SEP]
[CLS] аудируемого лица ; [SEP]
[CLS] АУДИТОР [SEP]
[CLS] Бухгалтерский баланс [SEP]
[CLS] Кады [SEP]
[CLS] АКТИВ [SEP]
[CLS] 1. ВНЕОБОРОТНЫЕ АКТИВЫ [SEP]
[CLS] ( ИСКЕ ) [SEP]
[CLS] 1. ОБОРОТНЫЕ АКТИВЫ [SEP]
[CLS] ПАССИВ [SEP]
[CLS] 11. КАПИТАЛ И РЕЗЕРВЫ [SEP]
[CLS] Отчет о финансовых результатах [SEP]
[CLS] за Январь - Декабрь 2023 г. Коды [SEP]
[CLS] Отчет об изменениях капитала [SEP]
[CLS] за Январь - Декабрь 2023 г. Коды [SEP]
[CLS] 1. Движение капитала [SEP]
[CLS]. 2. Корректировки в связи с изменением учетной политики и исправлением ошибок [SEP]
[CLS] 3. Чистые активы [SEP]
[CLS] Наименование показателя [SEP]
[CLS] Отчет о движении денежны

In [131]:
torch.cuda.empty_cache()

In [163]:
ouputs[1]

IndexError: index 1 is out of bounds for dimension 0 with size 1